## Perform Imports

In [ ]:
# import dependencies
from bs4 import BeautifulSoup
from os.path import join
from math import sin, cos, pi

In [ ]:
# import confidential information
from sys import path
path.insert(0, "..")
from config import pc_raw_export
from reference_values import probe_dict

## Define Functions

## Read Raw Data

In [ ]:
xml_file_path = join(pc_raw_export, "7005132 - A.xml")

# open the xml file
with open(xml_file_path, "r", encoding = "utf-8") as file:
    file_contents = file.read()

# create the beautifulsoup object
bs = BeautifulSoup(file_contents, "xml")

In [ ]:
# initialize the output dictionary
output_dict = {
    "drawing": "empty",
    "revision": "empty",
    "fixture": "empty",
    "nx": 0,
    "py": 0,
    "px": 0,
    "ny": 0
}

In [ ]:
# get a list of all the command features
feature_list = bs.find_all("Command")

# initialize parameters
current_probe = ""
probe_length = 0.0
final_measurement = False
measurement_index = 0
measurement_count = len([x for x in feature_list if "(CONTACT)" in str(x)])

# extract elements
for feature in feature_list:
    feature_type = str(feature.get("Type"))
    if feature_type == "File Header":
        output_dict["drawing"] = str(feature.find_all("DataField")[0].get("Value"))
        output_dict["revision"] = str(feature.find_all("DataField")[1].get("Value"))
    elif feature_type == "Recall Alignment":
        output_dict["fixture"] = str(feature.find_all("DataField")[2].get("Value"))
    elif feature_type == "Load Probe":
        current_probe = str(feature.find("DataField").get("Value"))
        probe_length = probe_dict[current_probe]
    elif feature_type == "Set Active Tip":
        tip_str = str(feature.find_all("DataField")[0].get("Value"))
        A, B = [float(x) * pi / 180 for x in tip_str[3:].split("B")]
        
        distance = probe_length * sin(A)
        dist_x = round(distance * sin(B), 4)
        dist_y = round(-distance * cos(B), 4)
        
        if dist_x >= 0:
            if output_dict["nx"] < abs(dist_x):
                output_dict["nx"] = abs(dist_x)
        else:
            if output_dict["px"] < abs(dist_x):
                output_dict["px"] = abs(dist_x)
        if dist_y >= 0:
            if output_dict["ny"] < abs(dist_y):
                output_dict["ny"] = abs(dist_y)
        else:
            if output_dict["py"] < abs(dist_y):
                output_dict["py"] = abs(dist_y)
        
    elif "(CONTACT)" in feature_type:
        measurement_index += 1
        if measurement_index == measurement_count:
            break

output_dict

## Explore the Datasets